In [ ]:
import geopandas as gpd
import json
import os

# Path to your file
file_path = r"C:\Users\Gebruiker\OneDrive\DesirMED info\apps\multi-gis-framework\database\EU_education.geojson"

# Step 1: Try reading just the metadata using json (not loading geometry)
try:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(" JSON loaded successfully.")
    print(f"Number of features: {len(data['features'])}")
    print("Sample properties of first feature:")
    print(data["features"][0]["properties"])
except Exception as e:
    print(" Error reading JSON file:", e)

print("\n" + "="*50 + "\n")

# Step 2: Try reading with GeoPandas but ignoring geometry
try:
    df = gpd.read_file(file_path, ignore_geometry=True)
    print(" GeoDataFrame (no geometry) loaded.")
    print("Columns:", df.columns.tolist())
    print("First rows:")
    print(df.head())
except Exception as e:
    print(" Error loading GeoDataFrame (no geometry):", e)

print("\n" + "="*50 + "\n")

# Step 3: Try reading the full GeoDataFrame
try:
    gdf = gpd.read_file(file_path)
    print(" Full GeoDataFrame loaded.")
    print("CRS:", gdf.crs)
    print("Geometry type:", gdf.geom_type.unique())
    print("First rows:")
    print(gdf.head())
except Exception as e:
    print(" Error loading full GeoDataFrame:", e)


In [ ]:
import geopandas as gpd
import os

# Define input and output paths
folder_path = r"C:\Users\Gebruiker\OneDrive\DesirMED info\apps\multi-gis-framework\database"
input_file = os.path.join(folder_path, "EU_education.geojson")
output_file = os.path.join(folder_path, "EU_education.feather")

# Load GeoJSON
gdf = gpd.read_file(input_file)

# Save as Feather
gdf.to_feather(output_file)

print(f" Feather file saved at: {output_file}")


In [ ]:
import geopandas as gpd
import math
import os

# Configuration
input_path = r"C:\Users\Gebruiker\OneDrive\DesirMED info\apps\multi-gis-framework\database\EU_education.geojson"
output_folder = r"C:\Users\Gebruiker\OneDrive\DesirMED info\apps\multi-gis-framework\database"
base_filename = "EU_education"
records_per_split = 12366
output_format = "geojson"  # or "feather"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the full GeoJSON
gdf = gpd.read_file(input_path)

# Calculate number of splits
num_splits = math.ceil(len(gdf) / records_per_split)

# Perform the splits
for i in range(num_splits):
    start = i * records_per_split
    end = (i + 1) * records_per_split
    gdf_chunk = gdf.iloc[start:end]

    part_number = i + 1
    if output_format == "geojson":
        out_file = os.path.join(output_folder, f"{base_filename}{part_number}.geojson")
        gdf_chunk.to_file(out_file, driver="GeoJSON")
    elif output_format == "feather":
        out_file = os.path.join(output_folder, f"{base_filename}{part_number}.feather")
        gdf_chunk.to_feather(out_file)

    print(f"Saved: {out_file}")


In [7]:
import os
import geopandas as gpd
import pandas as pd

# Set your folder containing the parts
input_folder = r"C:\Users\Gebruiker\OneDrive\DesirMED info\apps\multi-gis-framework\database"
output_filename = "EU_education_merged.geojson"
output_path = os.path.join(input_folder, output_filename)

# Find all part files (e.g., EU_education_part1.geojson, etc.)
part_files = sorted([
    f for f in os.listdir(input_folder)
    if f.startswith("EU_education") and f.endswith(".geojson") and "merged" not in f
])

print(f"Found {len(part_files)} split files to merge:")
for f in part_files:
    print(" -", f)

if not part_files:
    print(" No split files found. Please check the folder and naming.")
else:
    # Load and concatenate
    gdf_parts = []
    for f in part_files:
        path = os.path.join(input_folder, f)
        try:
            gdf = gpd.read_file(path)
            gdf_parts.append(gdf)
        except Exception as e:
            print(f"⚠️ Error reading {f}: {e}")

    if gdf_parts:
        merged_gdf = gpd.GeoDataFrame(pd.concat(gdf_parts, ignore_index=True), crs=gdf_parts[0].crs)
        merged_gdf.to_file(output_path, driver="GeoJSON")
        print(f" Merged file saved as: {output_path}")
    else:
        print(" No valid GeoDataFrames to merge.")


Found 27 split files to merge:
 - EU_education1.geojson
 - EU_education10.geojson
 - EU_education11.geojson
 - EU_education12.geojson
 - EU_education13.geojson
 - EU_education14.geojson
 - EU_education15.geojson
 - EU_education16.geojson
 - EU_education17.geojson
 - EU_education18.geojson
 - EU_education19.geojson
 - EU_education2.geojson
 - EU_education20.geojson
 - EU_education21.geojson
 - EU_education22.geojson
 - EU_education23.geojson
 - EU_education24.geojson
 - EU_education25.geojson
 - EU_education26.geojson
 - EU_education27.geojson
 - EU_education3.geojson
 - EU_education4.geojson
 - EU_education5.geojson
 - EU_education6.geojson
 - EU_education7.geojson
 - EU_education8.geojson
 - EU_education9.geojson
 Merged file saved as: C:\Users\Gebruiker\OneDrive\DesirMED info\apps\multi-gis-framework\database\EU_education_merged.geojson
